In [ ]:
!nvidia-smi

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split

In [ ]:
!pip install wget
import wget
wget.download('https://storage.googleapis.com/wandb_datasets/nature_12K.zip')
!unzip /content/nature_12K.zip

In [ ]:
!pip install --upgrade wandb
!wandb login aa5afea12b4fda1e7f8310b597eb17c73d1176d2 #my API key for wandb login 
import wandb

# **Dataset creation(augmentation)**

In [ ]:
def dataset_creation(data_augmentation=True):
  if data_augmentation:
    train_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
        transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #These values are standard values used in imagenet competition
    ])
  else:
        train_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #These values are standard values used in imagenet competition
    ])
  
  val_transforms = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      #These values are standard values used in imagenet competition
  ])

  # Create training and validation datasets
  train_dataset = ImageFolder('/content/inaturalist_12K/train', transform=train_transforms)
  train_size = int(0.8 * len(train_dataset))
  val_size = len(train_dataset) - train_size
  train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
  test_dataset = ImageFolder('/content/inaturalist_12K/val', transform=val_transforms)
   
  # Create training and validation data loaders
  BATCH_SIZE=32
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
  val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
  return train_loader,val_loader

train_loader,val_loader=dataset_creation()
#BELOW LINES USED FOR DEBUGGING
'''
print(f"Data loader: {train_loader,val_loader}")
print(f"Length of train dataset loader:{len(train_loader)}")
train_features_batch,train_labels_batch=next(iter(train_loader))
train_features_batch.shape,train_labels_batch.shape'''

Data loader: (<torch.utils.data.dataloader.DataLoader object at 0x7fb8d72cdd90>, <torch.utils.data.dataloader.DataLoader object at 0x7fb8d72cde50>)
Length of train dataset loader:250


(torch.Size([32, 3, 224, 224]), torch.Size([32]))

# **VGG model**
Tried training on it, but the model is very bulky and highly dependent on hyperparameter tuning

In [ ]:
'''# # vgg16 model
def vgg(f_percent):
    vgg16_model = models.vgg16_bn(pretrained=True)

    counttotal = 0
    total_layers_freezed = f_percent 
    for param in vgg16_model.features.parameters():
        param.requires_grad = True
        counttotal += 1
    count = 0
    for param in vgg16_model.features.parameters():
        if count<int(total_layers_freezed*counttotal):
            param.requires_grad = False
            count+=1

    num_features = vgg16_model.classifier[6].in_features
    features = list(vgg16_model.classifier.children())[:-1]
    features.extend([nn.Linear(num_features, 10)])
    vgg16_model.classifier = nn.Sequential(*features)

    return vgg16_model'''

# **EfficientVetV2**
you need to install timm to use EfficientNet in PyTorch because timm provides PyTorch implementations of the EfficientNet models

In [ ]:
!pip install timm

In [ ]:
import torch
import timm

def efficientnet(freeze_percent):
    # Load the EfficientNetV2 model with pre-trained weights
    model = timm.create_model('tf_efficientnetv2_s', pretrained=True)
    
    # Freeze a percentage of layers
    count_total = sum(1 for _ in model.parameters())
    count = 0
    for param in model.parameters():
        if count < int(freeze_percent * count_total):
            param.requires_grad = False
            count += 1
        else:
            break
    
    # Replace the last layer with a new one for the specified number of classes
    num_features = model.classifier.in_features
    model.classifier = nn.Linear(num_features, 10)
    
    return model

# **Sweeping over hyperparameters**

In [ ]:
sweep_config = {
    'method': 'grid', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'freeze_percent': {
            'values': [0.25]
        },        
        'learning_rate':{
            'values':[0.0001,0.0003]
        },
        'beta1':{
            'values':[0.9,0.93,0.95]
        }
        
    }
}

sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS6910-Assignment2")

def sweep_train():
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'freeze_percent':0.25,
      'learning_rate':0.0003,
      'beta1':0.93,
  }

  # Initialize a new wandb run
  wandb.init(project='CS6910-Assignment2', entity='shreyashgadgil007',config=config_defaults)
  wandb.run.name = 'fp:'+ str(wandb.config.freeze_percent)+' ;lr:'+str(wandb.config.learning_rate)+ ' ;beta1:'+str(wandb.config.beta1)

  
  config = wandb.config
  freeze_percent = config.freeze_percent
  learning_rate = config.learning_rate
  beta1 = config.beta1
  # Model training here
  #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = efficientnet(freeze_percent).to(device)
  train_loader,val_loader=dataset_creation()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(),lr=learning_rate, betas=(beta1,0.9999))#using adam
  num_epochs = 10
  for epoch in range(num_epochs):
          # Set to training mode
          model.train()

          running_loss = 0.0
          running_corrects = 0
          for inputs, labels in train_loader:
              inputs, labels = inputs.to(device), labels.to(device)

              # Zero the parameter gradients
              optimizer.zero_grad()

              # Forward pass
              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)
              
              # Backward pass and optimize
              loss.backward()
              optimizer.step()

              # Statistics
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

          train_epoch_loss = running_loss / len(train_loader.dataset)
          train_epoch_acc = (running_corrects.double() / len(train_loader.dataset))*100

          # Evaluate on validation set
          model.eval()

          running_loss = 0.0
          running_corrects = 0
          with torch.no_grad():
              for inputs, labels in val_loader:
                  inputs, labels = inputs.to(device), labels.to(device)

                  # Forward pass
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)

                  # Statistics
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

          val_epoch_loss = running_loss / len(val_loader.dataset)
          val_epoch_acc = (running_corrects.double() / len(val_loader.dataset))*100
          
          print(f"Epoch {epoch+1}/{num_epochs}--> Training_Loss:{train_epoch_loss:.2f}; Train_Accuracy:{train_epoch_acc:.2f}; Validation_Loss:{val_epoch_loss:.2f}; Val_Accuracy:{val_epoch_acc:.2f}")
          wandb.log({"train_loss":train_epoch_loss,"train_accuracy": train_epoch_acc,"val_loss":val_epoch_loss,"val_accuracy":val_epoch_acc},)
          if epoch==num_epochs-1:
            torch.cuda.empty_cache()


wandb.agent(sweep_id, function=sweep_train, count=6)

Create sweep with ID: 2ju5fl58
Sweep URL: https://wandb.ai/shreyashgadgil007/CS6910-Assignment2/sweeps/2ju5fl58


wandb: Agent Starting Run: njjqrcck with config:
wandb: 	beta1: 0.9
wandb: 	freeze_percent: 0.25
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: shreyashgadgil007. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-effv2-weights/tf_efficientnetv2_s-eb54923e.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnetv2_s-eb54923e.pth


Epoch 1/10--> Training_Loss:1.30; Train_Accuracy:60.71; Validation_Loss:0.74; Val_Accuracy:76.50
Epoch 2/10--> Training_Loss:0.66; Train_Accuracy:78.97; Validation_Loss:0.61; Val_Accuracy:81.00
Epoch 3/10--> Training_Loss:0.48; Train_Accuracy:84.85; Validation_Loss:0.62; Val_Accuracy:80.45
Epoch 4/10--> Training_Loss:0.35; Train_Accuracy:89.05; Validation_Loss:0.61; Val_Accuracy:80.40
Epoch 5/10--> Training_Loss:0.28; Train_Accuracy:90.69; Validation_Loss:0.63; Val_Accuracy:80.40
Epoch 6/10--> Training_Loss:0.23; Train_Accuracy:92.37; Validation_Loss:0.65; Val_Accuracy:80.60
Epoch 7/10--> Training_Loss:0.18; Train_Accuracy:94.25; Validation_Loss:0.70; Val_Accuracy:80.40
Epoch 8/10--> Training_Loss:0.14; Train_Accuracy:95.76; Validation_Loss:0.70; Val_Accuracy:81.15
Epoch 9/10--> Training_Loss:0.13; Train_Accuracy:96.12; Validation_Loss:0.72; Val_Accuracy:80.35
Epoch 10/10--> Training_Loss:0.12; Train_Accuracy:96.24; Validation_Loss:0.74; Val_Accuracy:80.60


train_accuracy,▁▅▆▇▇▇████
train_loss,█▄▃▂▂▂▁▁▁▁
val_accuracy,▁█▇▇▇▇▇█▇▇
val_loss,█▁▂▁▂▄▆▆▇█
train_accuracy,96.23703
train_loss,0.11944
val_accuracy,80.6
val_loss,0.73696


...

wandb: Agent Starting Run: cshp7j7o with config:
wandb: 	beta1: 0.93
wandb: 	freeze_percent: 0.25
wandb: 	learning_rate: 0.0003
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/10--> Training_Loss:1.08; Train_Accuracy:65.28; Validation_Loss:0.78; Val_Accuracy:74.70
Epoch 2/10--> Training_Loss:0.64; Train_Accuracy:79.35; Validation_Loss:0.72; Val_Accuracy:77.40
Epoch 3/10--> Training_Loss:0.50; Train_Accuracy:84.07; Validation_Loss:0.76; Val_Accuracy:75.80
Epoch 4/10--> Training_Loss:0.38; Train_Accuracy:87.45; Validation_Loss:0.73; Val_Accuracy:77.90
Epoch 5/10--> Training_Loss:0.30; Train_Accuracy:89.96; Validation_Loss:0.79; Val_Accuracy:77.45
Epoch 6/10--> Training_Loss:0.26; Train_Accuracy:91.42; Validation_Loss:0.80; Val_Accuracy:77.35
Epoch 7/10--> Training_Loss:0.22; Train_Accuracy:92.56; Validation_Loss:0.79; Val_Accuracy:77.05
Epoch 8/10--> Training_Loss:0.19; Train_Accuracy:93.61; Validation_Loss:0.87; Val_Accuracy:77.25
Epoch 9/10--> Training_Loss:0.16; Train_Accuracy:94.70; Validation_Loss:0.92; Val_Accuracy:76.65
Epoch 10/10--> Training_Loss:0.16; Train_Accuracy:94.81; Validation_Loss:0.91; Val_Accuracy:76.55


train_accuracy,▁▄▅▆▇▇▇███
train_loss,█▅▄▃▂▂▁▁▁▁
val_accuracy,▁▇▃█▇▇▆▇▅▅
val_loss,▃▁▃▁▃▄▄▇██
train_accuracy,94.81185
train_loss,0.15982
val_accuracy,76.55
val_loss,0.90927


wandb: Agent Starting Run: xegvjib4 with config:
wandb: 	beta1: 0.95
wandb: 	freeze_percent: 0.25
wandb: 	learning_rate: 0.0001
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/10--> Training_Loss:1.30; Train_Accuracy:60.46; Validation_Loss:0.73; Val_Accuracy:77.00
Epoch 2/10--> Training_Loss:0.66; Train_Accuracy:78.83; Validation_Loss:0.61; Val_Accuracy:80.45
Epoch 3/10--> Training_Loss:0.48; Train_Accuracy:84.57; Validation_Loss:0.57; Val_Accuracy:81.60
Epoch 4/10--> Training_Loss:0.37; Train_Accuracy:88.39; Validation_Loss:0.65; Val_Accuracy:79.30
